In [2]:
import requests
import datetime
import base64
import webbrowser
from http.server import HTTPServer, BaseHTTPRequestHandler
from urllib.parse import urlencode, urlparse, parse_qsl

In [3]:
client_id = 'e9a3d54e253d4efc91ada2f4cdd67cfc'
client_secret = '74d8228a44694380a45feafaa516ac7d'

In [4]:
client_creds = f"{client_id}:{client_secret}"

In [5]:
client_creds_b64 = base64.b64encode(client_creds.encode())
print(client_creds_b64)

b'ZTlhM2Q1NGUyNTNkNGVmYzkxYWRhMmY0Y2RkNjdjZmM6NzRkODIyOGE0NDY5NDM4MGE0NWZlYWZhYTUxNmFjN2Q='


In [6]:
# Clase para crear un servidor local y obtener el codigo de autorizacion
# Obtenido de https://github.com/plamere/spotipy/blob/master/spotipy/oauth2.py

In [9]:
class RequestHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        query_s = urlparse(self.path).query
        form = dict(parse_qsl(query_s))

        self.send_response(200)
        self.send_header("Content-Type", "text/html")
        self.end_headers()

        if "code" in form:
            self.server.auth_code = form["code"]
            self.server.error = None
            status = "successful"
        elif "error" in form:
            self.server.error = form["error"]
            self.server.auth_code = None
            status = "failed ({})".format(form["error"])
        else:
            self._write("<html><body><h1>Invalid request</h1></body></html>")
            return

        self._write("""
        <html>
        <script>
        window.close()
        </script>
        <body>
        <h1>Authentication status: {}</h1>
        This window can be closed.
        </body>
        </html>
        """.format(status))

    def _write(self, text):
        return self.wfile.write(text.encode("utf-8"))

    def log_message(self, format, *args):
        return


def start_local_http_server(port, handler=RequestHandler):
    server = HTTPServer(("localhost", port), handler)
    server.allow_reuse_address = True
    server.auth_code = None
    server.error = None
    return server

In [ ]:
def open_auth_url(self):
    auth_url = self.get_authorize_url()
    try:
        webbrowser.open(auth_url)
        logger.info("Opened %s in your browser", auth_url)
    except webbrowser.Error:
        logger.error("Please navigate here: %s", auth_url)

In [10]:
def get_auth_response_local_server(redirect_port):
        server = start_local_http_server(redirect_port)
        #self._open_auth_url()
        server.handle_request()

        if server.auth_code is not None:
            print(server.auth_code)
            return server.auth_code
        elif server.error is not None:
            raise Exception("Received error from OAuth server: {}".format(server.error))
        else:
            raise Exception("Server listening on localhost has not been accessed")

In [11]:
# Autorizacion de la app
def get_authorize_url(self):
    endpoint = "https://accounts.spotify.com/authorize"
    local_uri = "http://localhost:8000"
    auth_data = {
        "client_id": client_id,
        "response_type": "code",
        "redirect_uri": local_uri
    }
    data = urlencode(auth_data)
    auth_url = f"{endpoint}?{data}"
    return auth_url

https://accounts.spotify.com/authorize?client_id=e9a3d54e253d4efc91ada2f4cdd67cfc&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8000


In [12]:
r = requests.get(auth_url)
print(r.status_code)
auth_code = get_auth_response_local_server(redirect_port=8000)

200
AQBLGr6okjTSF1O-qzeEcABrez8YBvSnDMPb8CARbH0RaQtonQQ_OAo_9LHhJ2zJKQ_A2B24gtnbZybzcsFx7zQ_altbKlQdrn8KWCCi4ffNpLvdKtXW3DGEvDtIBhg3TJq6yd6aitwkzQyHuEknPuSM7ju3kC7lVQ


'AQBLGr6okjTSF1O-qzeEcABrez8YBvSnDMPb8CARbH0RaQtonQQ_OAo_9LHhJ2zJKQ_A2B24gtnbZybzcsFx7zQ_altbKlQdrn8KWCCi4ffNpLvdKtXW3DGEvDtIBhg3TJq6yd6aitwkzQyHuEknPuSM7ju3kC7lVQ'

In [ ]:
# Solicitar token de acceso y de renovacion

In [14]:
token_url = "https://accounts.spotify.com/api/token"
token_data = {
    "grant_type": "authorization_code",
    "code": "AQBLGr6okjTSF1O-qzeEcABrez8YBvSnDMPb8CARbH0RaQtonQQ_OAo_9LHhJ2zJKQ_A2B24gtnbZybzcsFx7zQ_altbKlQdrn8KWCCi4ffNpLvdKtXW3DGEvDtIBhg3TJq6yd6aitwkzQyHuEknPuSM7ju3kC7lVQ",
    "redirect_uri": local_uri
}
token_headers = {
    "Authorization": f"Basic {client_creds_b64.decode()}" # <base64 encoded client_id:client_secret>
}

In [15]:
r = requests.post(token_url, data=token_data, headers=token_headers)
print(r.json())
valid_request = r.status_code in range(200, 299)

{'access_token': 'BQDzgChkCn_hkY69or2QoOfUCDewuH9pvo9BLdf0z8qlnBxuiUM9rNpxgM3QEV1FKrOd5oPWqzPPi85K7Xs-hvbpurrFh6hrAO_ZP2L5bh5tXaqX6-CJ80uWv98fMYfLlcD2_AdHjllE2yh4RNx8fgA_pSGucO3teJxkKg', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'AQB5dZuTnjOac7gqSdXGc4YXl2X3zAss3j3YMrmHD3VR5Q_8SsTdz0yFOpPalzWKGXVCAKmiVwdhmpJgtkYw51p82mvVLzUUSBe9GF2QXXvhdXo0qR2v2TXmGB1X7a7_-ic', 'scope': ''}


In [18]:
if valid_request:
    token_response_data = r.json()
    now = datetime.datetime.now()
    access_token = token_response_data['access_token']
    refresh_token = token_response_data['refresh_token']
    print(refresh_token)
    expires_in = token_response_data['expires_in'] # seconds
    expires = now + datetime.timedelta(seconds=expires_in)
    did_expire = expires < now

AQB5dZuTnjOac7gqSdXGc4YXl2X3zAss3j3YMrmHD3VR5Q_8SsTdz0yFOpPalzWKGXVCAKmiVwdhmpJgtkYw51p82mvVLzUUSBe9GF2QXXvhdXo0qR2v2TXmGB1X7a7_-ic
